# 5. Extraindo *Features* de LLMs

In [2]:
# Importando as bibliotecas para lidar com LLMs
import pandas as pd
import ollama
import re
import requests
import time

In [70]:
# Caminho do CSV original
input_path = "/content/enem_data_embeddings_100.pkl"
output_path = "llamma_extraction.pkl"

In [4]:
# Carregar CSV
df = pd.read_pickle(input_path)
df.shape

(270, 16)

In [5]:
n = 6
tamanho = len(df) // n  # 45

# Dividindo em 6 DataFrames separados
df1 = df.iloc[0*tamanho : 1*tamanho].reset_index(drop=True)
df2 = df.iloc[1*tamanho : 2*tamanho].reset_index(drop=True)
df3 = df.iloc[2*tamanho : 3*tamanho].reset_index(drop=True)
df4 = df.iloc[3*tamanho : 4*tamanho].reset_index(drop=True)
df5 = df.iloc[4*tamanho : 5*tamanho].reset_index(drop=True)
df6 = df.iloc[5*tamanho : 6*tamanho].reset_index(drop=True)

# Verificando os tamanhos
print(len(df1), len(df2), len(df3), len(df4), len(df5), len(df6))

45 45 45 45 45 45


---
## 5.1. Configurando modelo LLM

In [6]:
# Definindo prompt para o modelo
prompt = """Responda à seguinte questão:

    Enunciado:
    {enunciado}

    Alternativas:
    {alternativas}

    Qual a alternativa correta? Responda explicitamente apenas com a respectiva letra (A, B, C, D ou E). (não inclua coisas como "É a letra A", quero apenas o caractere da letra)"""

### 5.1.1 LLama 3.2

In [62]:
LLAMA_API_KEY = "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"  # 🔒 Substitua pela sua chave
LLAMA_API_URL = "https://openrouter.ai/api/v1/chat/completions"
MODEL = "meta-llama/llama-3.2-3b-instruct:free"

In [63]:
def ask_llama(enunciado, alternativas, stream=False):
    """
    Envia a pergunta ao modelo Llama. Faz uma única requisição.

    Parâmetros:
    - enunciado: texto da pergunta
    - alternativas: opções de resposta
    - stream: se True, imprime chunks em tempo real
    """
    headers = {
        "Authorization": f"Bearer {LLAMA_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": MODEL,
        "messages": [
            {"role": "user", "content": prompt.format(enunciado=enunciado, alternativas=alternativas)}
        ],
        "stream": stream,
        "max_tokens": 2048
    }

    try:
        response = requests.post(LLAMA_API_URL, headers=headers, json=payload)

        if response.status_code == 429:
            return f"Erro 429: Rate limit excedido. Tente novamente mais tarde."

        response.raise_for_status()

        if stream:
            for chunk in response.iter_lines():
                if chunk:
                    print("Chunk recebido:", chunk.decode('utf-8'))
            return None
        else:
            full_response = response.json()
            return full_response['choices'][0]['message']['content']

    except requests.exceptions.HTTPError as http_err:
        return f"Erro HTTP na API: {http_err}"
    except Exception as e:
        return f"Erro na API: {e}"


### 3.1.2 Deepseek

In [ ]:
# Configurações da API
DEEPSEEK_API_KEY = "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"  # 🔒 Substitua pela sua chave
DEEPSEEK_API_URL = "https://openrouter.ai/api/v1/chat/completions"
MODEL = "deepseek/deepseek-v3-base:free"

In [ ]:
# modelo utilizado deepseek/deepseek-v3-base:free


def ask_deepseek(enunciado, alternativas, stream = False):

    headers = {
        "Authorization": f"Bearer {DEEPSEEK_API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": MODEL,
        "messages": [{"role": "user", "content": prompt.format(enunciado=enunciado, alternativas=alternativas)}],
        "stream": stream,  # Habilita resposta em tempo real se True
        "max_tokens": 2048  # Ajuste conforme necessário
    }

    try:
        response = requests.post(DEEPSEEK_API_URL, headers=headers, json=payload)
        response.raise_for_status()  # Verifica erros HTTP

        if stream:
            # Processamento para streaming (opcional)
            for chunk in response.iter_lines():
                if chunk:
                    print("Chunk recebido:", chunk.decode('utf-8'))
        else:
            full_response = response.json()
            return full_response['choices'][0]['message']['content']

    except Exception as e:
        return f"Erro na API: {str(e)}"


---
## 5.2. Aplicando Perguntas aos LLMs

In [20]:
# df1["resposta_llamma"] = df1[["enunciado", "alternativas"]].apply(
#     lambda row: ask_llama(row["enunciado"], row["alternativas"]), axis=1
# )


# df1.loc[40, "resposta_llamma"] = df1.loc[40:, ["enunciado", "alternativas"]].apply(
#     lambda row: ask_llama(row["enunciado"], row["alternativas"]), axis=1
# )


# df1.loc[38:40, "resposta_llamma"] = df1.loc[38:40:, ["enunciado", "alternativas"]].apply(
#     lambda row: ask_llama(row["enunciado"], row["alternativas"]), axis=1
# )

### DF1 COLETADO COMPLETO


In [30]:
# df2["resposta_llamma"] = df2[["enunciado", "alternativas"]].apply(
#     lambda row: ask_llama(row["enunciado"], row["alternativas"]), axis=1
# )

# mask_erro = df2["resposta_llamma"].str.contains("Erro", na=False)

# df2.loc[mask_erro, "resposta_llamma"] = df1.loc[mask_erro, ["enunciado", "alternativas"]].apply(
#     lambda row: ask_llama(row["enunciado"], row["alternativas"]), axis=1
# )


# df2.loc[37:, "resposta_llamma"] = df2.loc[37:, ["enunciado", "alternativas"]].apply(
#     lambda row: ask_llama(row["enunciado"], row["alternativas"]), axis=1
# )


### DF2 COLETADO COMPLETO


In [39]:
# df3["resposta_llamma"] = df3[["enunciado", "alternativas"]].apply(
#     lambda row: ask_llama(row["enunciado"], row["alternativas"]), axis=1
# )

# df3.loc[30:, "resposta_llamma"] = df3.loc[30:, ["enunciado", "alternativas"]].apply(
#     lambda row: ask_llama(row["enunciado"], row["alternativas"]), axis=1
# )

### DF3 COLETADO COMPLETO

In [46]:
# df4["resposta_llamma"] = df4[["enunciado", "alternativas"]].apply(
#     lambda row: ask_llama(row["enunciado"], row["alternativas"]), axis=1
# )

# df4.loc[27:, "resposta_llamma"] = df4.loc[27:, ["enunciado", "alternativas"]].apply(
#     lambda row: ask_llama(row["enunciado"], row["alternativas"]), axis=1
# )

### DF4 COLETADO COMPLETO

In [53]:
# df5["resposta_llamma"] = df5[["enunciado", "alternativas"]].apply(
#     lambda row: ask_llama(row["enunciado"], row["alternativas"]), axis=1
# )

# df5.loc[28:, "resposta_llamma"] = df5.loc[28:, ["enunciado", "alternativas"]].apply(
#     lambda row: ask_llama(row["enunciado"], row["alternativas"]), axis=1
# )



### DF5 COLETADO COMPLETO

In [64]:
# df6["resposta_llamma"] = df6[["enunciado", "alternativas"]].apply(
#     lambda row: ask_llama(row["enunciado"], row["alternativas"]), axis=1
# )

# df6.loc[23:, "resposta_llamma"] = df6.loc[23:, ["enunciado", "alternativas"]].apply(
#     lambda row: ask_llama(row["enunciado"], row["alternativas"]), axis=1
# )

# df6.loc[42:, "resposta_llamma"] = df6.loc[42:, ["enunciado", "alternativas"]].apply(
#     lambda row: ask_llama(row["enunciado"], row["alternativas"]), axis=1
# )

### DF6 COLETADO COMPLETO

In [66]:
df_completo = pd.concat([df1, df2, df3, df4, df5, df6], ignore_index=True)

In [68]:
df_completo.to_csv('enem_llama_extraction.csv')

In [69]:
df_completo["acerto_llamma"] = df_completo[["gabarito", "resposta_llamma"]].apply(
    lambda row: row["gabarito"] == row["resposta_llamma"], axis=1
)

# df_completo["acerto_deepseek"] = df_completo[["gabarito", "resposta_deepseek"]].apply(
#     lambda row: row["gabarito"] == row["resposta_deepseek"], axis=1
# )

---
## 5.3. Armazenando Resultado

In [71]:
# Salvar resultado
df_completo.to_pickle(output_path)
df_completo.to_csv('llama_extraction_csv.csv')
# pd.(df, output_path, index=False)

In [73]:
df_completo["acerto_llamma"].sum()

np.int64(104)

In [ ]:
df['acerto_deepseek'].sum()

---